## Import libraries

In [ ]:
#!pip install nvidia-modulus

In [1]:
# Required libraries
import os
import time
import torch
import numpy as np
import torch.nn as nn
import torch.optim as optim
import torch.profiler
import torch.nn.functional as F
from torch.cuda.amp import autocast, GradScaler

import torch.utils.data as data
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
from pytorch_lightning.callbacks import LearningRateMonitor, ModelCheckpoint
from torch.cuda.amp import autocast
import torch_geometric
import torch_geometric.nn as geom_nn
from torch_geometric.nn import (
    MessagePassing,
    GCNConv,
    global_mean_pool as gap,
    global_max_pool as gmp,
)
from torch_geometric.data import Data, Dataset, DataLoader
from physicsnemo.models.fno import FNO
#from modulus.models.fno import FNO
# Device setup
device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print(device)

/lustre/env/miniconda3/envs/NeurOCAST-4/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


cuda:0


##  Configuration

In [2]:
train_size = 0.85
val_size = 1-train_size
#test_size = 0.15
Batch_size = 4
learning_rate = 1e-4
weight_decay= 1e-5
epochs = 200
#num_workers = 2

## Preprocess the data

In [3]:
# Load input and output
x_npy = np.load('/lustre/gpu-lustre/code/Input/x_array_4_years_filtered_utc.npy')
y_npy = np.load('/lustre/gpu-lustre/code/Input/y_array_4_years_filtered_utc.npy')
time_npy = np.load('/lustre/gpu-lustre/code/Input/time_array_4_years_filtered_utc.npy')
#edge_index = torch.load("/lustre/code/BiasCorrection/Codes/edge_index.pt")


In [4]:
X = torch.tensor(x_npy)
Y = torch.tensor(y_npy)

X.shape, Y.shape, time_npy.shape

(torch.Size([3473, 19, 8, 186]), torch.Size([3473, 19, 186]), (3473, 186))

#### Randomley separate 16 stations for training

In [5]:
'''
import random

START = 0
END = 18 
COUNT = 3

selection_pool = range(START, END + 1)

random_numbers = random.sample(selection_pool, COUNT)

random_numbers
'''

'\nimport random\n\nSTART = 0\nEND = 18 \nCOUNT = 3\n\nselection_pool = range(START, END + 1)\n\nrandom_numbers = random.sample(selection_pool, COUNT)\n\nrandom_numbers\n'

In [6]:
# keep the three stations out for the test

X_16_all= X[:,[i for i in range(X.size(1)) if i not in [12, 5, 1]], :,:]
Y_16_all = Y[:,[i for i in range(Y.size(1)) if i not in [12, 5, 1]], :]

X_16_all.shape, Y_16_all.shape



(torch.Size([3473, 16, 8, 186]), torch.Size([3473, 16, 186]))

#### Training and validation using different years

In [7]:
time_npy

array([['2021-03-01T01:00:00.000000000', '2021-03-01T02:00:00.000000000',
        '2021-03-01T03:00:00.000000000', ...,
        '2021-03-08T16:00:00.000000000', '2021-03-08T17:00:00.000000000',
        '2021-03-08T18:00:00.000000000'],
       ['2021-03-01T07:00:00.000000000', '2021-03-01T08:00:00.000000000',
        '2021-03-01T09:00:00.000000000', ...,
        '2021-03-08T22:00:00.000000000', '2021-03-08T23:00:00.000000000',
        '2021-03-09T00:00:00.000000000'],
       ['2021-03-01T13:00:00.000000000', '2021-03-01T14:00:00.000000000',
        '2021-03-01T15:00:00.000000000', ...,
        '2021-03-09T04:00:00.000000000', '2021-03-09T05:00:00.000000000',
        '2021-03-09T06:00:00.000000000'],
       ...,
       ['2024-10-22T01:00:00.000000000', '2024-10-22T02:00:00.000000000',
        '2024-10-22T03:00:00.000000000', ...,
        '2024-10-29T16:00:00.000000000', '2024-10-29T17:00:00.000000000',
        '2024-10-29T18:00:00.000000000'],
       ['2024-10-22T07:00:00.000000000', '20

In [8]:
# Seperate the years
time_npy_str = time_npy.astype(str)

index_year = []
for year in [2021,2022,2023,2024]:
    index = next(i for i, row in enumerate(time_npy_str) if np.any(np.char.startswith(row, str(year))))
    index_year.append(index)

index_year.append(time_npy.shape[0])
print(index_year)

[0, 892, 1930, 2646, 3473]


In [9]:
YEARS = [2021, 2022, 2023, 2024]
TOTAL_DATA_POINTS = index_year[-1]

splits_by_test_year = {}

print("Performing Leave-One-Year-Out (LOYO) cross-validation splits using index masking...")

for k, test_year in enumerate(YEARS):
    # 1. Determine the indices for the current TEST year (k)
    test_start_index = index_year[k]
    test_end_index = index_year[k+1]-1
    
    # 2. Extract TEST data (Same as before)
    X_test = X[test_start_index:test_end_index, :, :, :]
    Y_test = Y[test_start_index:test_end_index, :, :]
    Time_test = time_npy[test_start_index:test_end_index, :]

    # 3. Create TRAIN/VALIDATION data by removing TEST indices using a boolean mask
    
    # Initialize a boolean mask to TRUE for every data point
    keep_mask = np.ones(TOTAL_DATA_POINTS, dtype=bool)

    # Set the indices corresponding to the test data range to FALSE (this removes them)
    # This is equivalent to removing the current year's indices from the dataset.
    keep_mask[test_start_index:test_end_index] = False
    
    # 4. Use the mask to select the TRAIN/VALIDATION data directly from the source arrays
    X_train_val = X_16_all[keep_mask]
    Y_train_val = Y_16_all[keep_mask]
    Time_train_val = time_npy[keep_mask]
    
    # 5. Store the full split in the main dictionary
    splits_by_test_year[test_year] = {
        'X_train_val': X_train_val,
        'Y_train_val': Y_train_val,
        'Time_train_val': Time_train_val,
        'X_test': X_test,
        'Y_test': Y_test,
        'Time_test': Time_test
    }
    
    print(f"Split completed for TEST YEAR {test_year}:")
    print(f"  Train/Val size: {X_train_val.shape[0]} samples")
    print(f"  Test size: {X_test.shape[0]} samples\n")


Performing Leave-One-Year-Out (LOYO) cross-validation splits using index masking...
Split completed for TEST YEAR 2021:
  Train/Val size: 2582 samples
  Test size: 891 samples

Split completed for TEST YEAR 2022:
  Train/Val size: 2436 samples
  Test size: 1037 samples

Split completed for TEST YEAR 2023:
  Train/Val size: 2758 samples
  Test size: 715 samples

Split completed for TEST YEAR 2024:
  Train/Val size: 2647 samples
  Test size: 826 samples



#### Define the edge index for training stations

In [10]:
center = (-70.83, 42.24) 
x_min, y_min = center[0] - 5, center[1] - 2

station_convert = []

for i in range(16):
    station = [(float(splits_by_test_year[2021]['X_train_val'][0,i,7,0])- x_min)/2, (float(splits_by_test_year[2021]['X_train_val'][0,i,6,0])- y_min)/2]
    station_convert.append(station)
station_convert = torch.from_numpy(np.array(station_convert)).float()

    # generate edge_index
row_a=[]
row_b=[]  
row_ix=[]
row_iy=[]
row_jx=[]
row_jy=[]
for i in range(16):
    for j in range(16):
        row_a.append(i)
        row_b.append(j)
        row_ix.append(station_convert[i,0])
        row_iy.append(station_convert[i,1])
        row_jx.append(station_convert[j,0])
        row_jy.append(station_convert[j,1])

edge_index=[row_a,row_b,row_ix,row_iy,row_jx,row_jy]
edge_index = torch.from_numpy(np.array(edge_index)).float().to(device)
edge_index.shape

torch.Size([6, 256])

#### Define the edge index for testing stations

In [11]:
center = (-70.83, 42.24) 
x_min, y_min = center[0] - 5, center[1] - 2

station_convert = []

for i in range(19):
    station = [(float(splits_by_test_year[2021]['X_test'][0,i,7,0])- x_min)/2, (float(splits_by_test_year[2021]['X_test'][0,i,6,0])- y_min)/2]
    station_convert.append(station)
station_convert = torch.from_numpy(np.array(station_convert)).float()

    # generate edge_index
row_a=[]
row_b=[]  
row_ix=[]
row_iy=[]
row_jx=[]
row_jy=[]
for i in range(19):
    for j in range(19):
        row_a.append(i)
        row_b.append(j)
        row_ix.append(station_convert[i,0])
        row_iy.append(station_convert[i,1])
        row_jx.append(station_convert[j,0])
        row_jy.append(station_convert[j,1])

edge_index_19=[row_a,row_b,row_ix,row_iy,row_jx,row_jy]
edge_index_19 = torch.from_numpy(np.array(edge_index_19)).float().to(device)
edge_index_19.shape

torch.Size([6, 361])

## Use a dataloader and train and save the model and test the model

In [12]:
# Define the model
from neurocast import NeurOCAST
from torch.optim.lr_scheduler import ReduceLROnPlateau
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")



In [13]:
from torch_geometric.data import DataLoader, Batch

def generate_data(x_tensor, y_tensor,edge_index, num_graphs, num_nodes, num_features):
    data_list = []
    for i in range(num_graphs):
        x = x_tensor[i, :, :, :]  
        y = y_tensor[i, :, :]  
        edge_index = edge_index
        # Create the graph data object
        data = Data(x=x, edge_index=edge_index, y=y)
        data_list.append(data)

    return data_list


In [14]:

for k, test_year in enumerate(YEARS):
    # -------------------------------
    # Split the data and define dataloader
    # Determine the indices for each split
    split_year = splits_by_test_year[test_year]

    # Retrieve the training data for this split:
    X_train_for_year_test = split_year['X_train_val']
    Y_train_for_year_test = split_year['Y_train_val']

    num_samples = X_train_for_year_test.shape[0]  
    # Generate shuffled indices
    shuffled_indices = np.random.permutation(num_samples)


    # Shuffle X and Y using the shuffled indices
    X_train_for_year_test = X_train_for_year_test[shuffled_indices]
    Y_train_for_year_test = Y_train_for_year_test[shuffled_indices]


    train_end = int(train_size * num_samples)
    
    # Split the data
    x_train0 = X_train_for_year_test[:train_end,:,:,:].numpy()
    y_train0 = Y_train_for_year_test[:train_end,:,:].numpy()

    x_val0= X_train_for_year_test[train_end:,:,:,:].numpy()
    y_val0 = Y_train_for_year_test[train_end:,:,:].numpy()

    # Normlaize the data using train data
    x_train0_normalized = np.empty([x_train0.shape[0],x_train0.shape[1], x_train0.shape[2], x_train0.shape[3]], dtype=float)

    for d in range(x_train0.shape[2]):
       x_mean, x_std = x_train0[:,:,d,:].mean(), x_train0[:,:,d,:].std()
       x_train0_normalized[:,:,d,:] = (x_train0[:,:,d,:] - x_mean)/x_std
    x_train = torch.tensor(x_train0_normalized, dtype = torch.float32)

    x_val0_normalized = np.empty([x_val0.shape[0], x_val0.shape[1], x_val0.shape[2], x_val0.shape[3]], dtype=float)

    for d in range(x_train0.shape[2]):
       x_mean, x_std = x_train0[:,:,d,:].mean(), x_train0[:,:,d,:].std()
       x_val0_normalized[:,:,d,:] = (x_val0[:,:,d,:] - x_mean)/x_std
    x_val = torch.tensor(x_val0_normalized, dtype = torch.float32)


    y_train0_mean, y_train0_std = y_train0.mean(), y_train0.std()

    y_train0_normalized = y_train0
    # - y_train0_mean)/y_train0_std
    y_train = torch.tensor(y_train0_normalized, dtype = torch.float32)

    y_val0_normalized = y_val0
    #y_val0_normalized = (y_val0 - y_train0_mean)/y_train0_std
    y_val = torch.tensor(y_val0_normalized, dtype = torch.float32)


    # Generate graph data for each set
    train_data = generate_data(x_train, y_train, edge_index, num_graphs=train_end, num_nodes=16, num_features=8)
    val_data = generate_data(x_val, y_val, edge_index,  num_graphs=num_samples-train_end, num_nodes=16, num_features=8)


    # Check data shapes 
    print(f"Train data for test year {test_year}: {len(train_data)} graphs")
    print(f"Validation data for test year {test_year}: {len(val_data)} graphs")

    # Create DataLoader for each dataset
    train_loader = DataLoader(train_data, batch_size=Batch_size, shuffle=True, collate_fn=torch_geometric.data.Batch.from_data_list)
    val_loader = DataLoader(val_data, batch_size=Batch_size, shuffle=False, collate_fn=torch_geometric.data.Batch.from_data_list)

    # -------------------------------

    model = NeurOCAST(
    input_channels=8,
    output_channels=1,
    base_width=48,
    base_modes=24,
    num_layers=3,
    width_scale=[1.0, 2, 0.5],
    mode_scale=[1, 0.5, 2],
    resample_strategy=[0.7, 0.5, 2.5],
    padding=10
    ).to(device)

    
    # Train the model
    optimizer = optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=weight_decay)
    #scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)


    loss_module = torch.nn.MSELoss()

    # AMP GradScaler
    scaler = GradScaler()

    for epoch in range(epochs):
        epoch_start_time = time.time()
        model.train()
    
        total_loss = 0.0
        for data in train_loader:
            data = data.to(device)
        
            optimizer.zero_grad()

            with autocast(enabled=True, dtype=torch.float16):

                data.edge_index = torch.tensor(data.edge_index)  
                output = model(
                    data.x, 
                    data.edge_index[:2, :],   
                    data.edge_index[2:, :]    
                )
                loss = loss_module(output, data.y)
            total_loss += loss.item()     

            # scaled backprop
            scaler.scale(loss).backward()
            scaler.step(optimizer)
            scaler.update()

    
        # Validation
        model.eval()
        val_loss = 0.0
        with torch.no_grad():
            for data in val_loader:
                data = data.to(device)
                with autocast():
                    data.edge_index = torch.tensor(data.edge_index)  
                    output = model(
                        data.x, 
                        data.edge_index[:2, :],
                        data.edge_index[2:, :]
                    )
                    loss = loss_module(output, data.y)
                val_loss += loss.item()

        avg_train_loss = total_loss / len(train_loader)
        avg_val_loss   = val_loss / len(val_loader)



        epoch_time = time.time() - epoch_start_time
        print(f"Epoch {epoch+1}/{epochs} | "
              f"Train Loss: {avg_train_loss:.4f} | "
              f"Val Loss: {avg_val_loss:.4f} | "
              f"Time: {epoch_time:.2f}s")

    torch.save(model.state_dict(), f"NeurOCAST_model_{test_year}.pth")


    # -------------------------------
    # Test the model

    # Retrieve the training data for this split:
    X_test_for_year_test = split_year['X_test']
    Y_test_for_year_test = split_year['Y_test']


    # Normlaize the data using train data
    x_test0_normalized = np.empty([X_test_for_year_test.shape[0], X_test_for_year_test.shape[1], X_test_for_year_test.shape[2], X_test_for_year_test.shape[3]], dtype=float)

    for d in range(x_train0.shape[2]):
       x_mean, x_std = x_train0[:,:,d,:].mean(), x_train0[:,:,d,:].std()
       x_test0_normalized[:,:,d,:] = (X_test_for_year_test[:,:,d,:] - x_mean)/x_std
    x_test = torch.tensor(x_test0_normalized, dtype = torch.float32)


    y_test0_normalized = Y_test_for_year_test
    #y_test0_normalized = (Y_20_2024_cleaned_Y - y_train0_mean)/y_train0_std
    y_test = torch.tensor(y_test0_normalized, dtype = torch.float32)


    # Generate graph data for each set
    test_data = generate_data(x_test, y_test, edge_index_19, num_graphs=x_test.shape[0], num_nodes=19, num_features=8)


    # Create DataLoader for each dataset
    test_loader = DataLoader(test_data, batch_size=Batch_size, shuffle=False, collate_fn=torch_geometric.data.Batch.from_data_list)

    def eval_model(model, data_loader):
        model.eval()  # Set model to eval mode
        all_preds = []
        all_targets = []

        with torch.no_grad():  # Deactivate gradients for the following code
            for data in test_loader:
                # Send data to device (GPU/CPU)
                data = data.to(torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
                #data = data.to(device)


                # Normalize x and y
                #data.x = normalize_x(data.x)  # Normalize per feature/channel separately
                #data.y = normalize_y(data.y)  # Normalize target values per graph
                # Forward pass to get predictions
                preds = model(data.x, data.edge_index[:2,:], data.edge_index[2:,:])  # Assuming model returns the predictions
                #preds = preds.squeeze(dim=1)
                all_preds.append(preds)
                all_targets.append(data.y)

        # Concatenate all predictions and true targets into one tensor
        all_preds = torch.cat(all_preds, dim=0)
        all_targets = torch.cat(all_targets, dim=0)

        # Calculate MSE between all predictions and true targets
        #mse_loss = F.mse_loss(all_preds, all_targets)
        mse_loss = loss_module(all_preds, all_targets)
        return mse_loss, all_preds, all_targets

    # Run the test
    result = eval_model(model, test_loader)

    # Reformat 
    output = result[1].reshape(X_test_for_year_test.shape[0], 19, 186)
    target = result[2].reshape(X_test_for_year_test.shape[0], 19, 186)

    # Save the output 
    Test_prediction = output.cpu().numpy()+X_test_for_year_test[:,:,0,:].numpy()
    Test_raw = X_test_for_year_test[:,:,0,:].numpy()
    Test_target = y_test.numpy()+X_test_for_year_test[:,:,0,:].numpy()

    np.save(f"Test_prediction_{test_year}.npy", Test_prediction)
    np.save(f"Test_target_{test_year}.npy", Test_target)
    np.save(f"Test_raw_{test_year}.npy", Test_raw)

Train data for test year 2021: 2194 graphs
Validation data for test year 2021: 388 graphs


/lustre/env/miniconda3/envs/NeurOCAST-4/lib/python3.10/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)
/tmp/ipykernel_5826/1752502388.py:93: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
/tmp/ipykernel_5826/1752502388.py:105: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=True, dtype=torch.float16):
/tmp/ipykernel_5826/1752502388.py:107: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data.edge_index = torch.tensor(data.edge_index)
/lustre/env/miniconda3/envs/NeurOCAST-4/lib/python3.10/site-packages/torch/backends/cuda/__init__.py:131: Use

Epoch 1/200 | Train Loss: 0.0317 | Val Loss: 0.0238 | Time: 198.81s


/tmp/ipykernel_5826/1752502388.py:105: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=True, dtype=torch.float16):
/tmp/ipykernel_5826/1752502388.py:107: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data.edge_index = torch.tensor(data.edge_index)


Epoch 2/200 | Train Loss: 0.0215 | Val Loss: 0.0159 | Time: 20.63s
Epoch 3/200 | Train Loss: 0.0163 | Val Loss: 0.0150 | Time: 20.56s
Epoch 4/200 | Train Loss: 0.0145 | Val Loss: 0.0130 | Time: 21.11s
Epoch 5/200 | Train Loss: 0.0132 | Val Loss: 0.0124 | Time: 20.60s
Epoch 6/200 | Train Loss: 0.0124 | Val Loss: 0.0116 | Time: 20.54s
Epoch 7/200 | Train Loss: 0.0118 | Val Loss: 0.0110 | Time: 20.58s
Epoch 8/200 | Train Loss: 0.0112 | Val Loss: 0.0106 | Time: 20.54s
Epoch 9/200 | Train Loss: 0.0109 | Val Loss: 0.0104 | Time: 20.56s
Epoch 10/200 | Train Loss: 0.0104 | Val Loss: 0.0103 | Time: 20.55s
Epoch 11/200 | Train Loss: 0.0101 | Val Loss: 0.0105 | Time: 20.56s
Epoch 12/200 | Train Loss: 0.0098 | Val Loss: 0.0097 | Time: 20.55s
Epoch 13/200 | Train Loss: 0.0095 | Val Loss: 0.0097 | Time: 20.56s
Epoch 14/200 | Train Loss: 0.0093 | Val Loss: 0.0102 | Time: 20.59s
Epoch 15/200 | Train Loss: 0.0092 | Val Loss: 0.0091 | Time: 20.53s
Epoch 16/200 | Train Loss: 0.0089 | Val Loss: 0.0091 | T

...

Train data for test year 2022: 2070 graphs
Validation data for test year 2022: 366 graphs


/lustre/env/miniconda3/envs/NeurOCAST-4/lib/python3.10/site-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)
/tmp/ipykernel_5826/1752502388.py:93: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
/tmp/ipykernel_5826/1752502388.py:105: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast(enabled=True, dtype=torch.float16):
/tmp/ipykernel_5826/1752502388.py:107: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  data.edge_index = torch.tensor(data.edge_index)
/tmp/ipykernel_5826/1752502388.py:128: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Pleas

Epoch 1/200 | Train Loss: 0.0409 | Val Loss: 0.0262 | Time: 19.42s
Epoch 2/200 | Train Loss: 0.0231 | Val Loss: 0.0211 | Time: 19.39s
Epoch 3/200 | Train Loss: 0.0207 | Val Loss: 0.0207 | Time: 19.57s
Epoch 4/200 | Train Loss: 0.0193 | Val Loss: 0.0191 | Time: 19.67s
Epoch 5/200 | Train Loss: 0.0180 | Val Loss: 0.0176 | Time: 19.60s
Epoch 6/200 | Train Loss: 0.0169 | Val Loss: 0.0177 | Time: 19.61s
Epoch 7/200 | Train Loss: 0.0160 | Val Loss: 0.0159 | Time: 19.56s
Epoch 8/200 | Train Loss: 0.0152 | Val Loss: 0.0156 | Time: 19.59s
Epoch 9/200 | Train Loss: 0.0143 | Val Loss: 0.0145 | Time: 19.48s
Epoch 10/200 | Train Loss: 0.0137 | Val Loss: 0.0138 | Time: 19.38s
Epoch 11/200 | Train Loss: 0.0132 | Val Loss: 0.0137 | Time: 19.39s
Epoch 12/200 | Train Loss: 0.0127 | Val Loss: 0.0134 | Time: 19.38s
Epoch 13/200 | Train Loss: 0.0120 | Val Loss: 0.0131 | Time: 19.43s
Epoch 14/200 | Train Loss: 0.0117 | Val Loss: 0.0121 | Time: 19.38s
Epoch 15/200 | Train Loss: 0.0112 | Val Loss: 0.0118 | Ti

/tmp/ipykernel_5826/1752502388.py:165: DeprecationWarning: __array__ implementation doesn't accept a copy keyword, so passing copy=False failed. __array__ must implement 'dtype' and 'copy' keyword arguments. To learn more, see the migration guide https://numpy.org/devdocs/numpy_2_0_migration_guide.html#adapting-to-changes-in-the-copy-keyword
  x_test0_normalized[:,:,d,:] = (X_test_for_year_test[:,:,d,:] - x_mean)/x_std
/tmp/ipykernel_5826/1752502388.py:171: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.detach().clone() or sourceTensor.detach().clone().requires_grad_(True), rather than torch.tensor(sourceTensor).
  y_test = torch.tensor(y_test0_normalized, dtype = torch.float32)


Train data for test year 2023: 2344 graphs
Validation data for test year 2023: 414 graphs
Epoch 1/200 | Train Loss: 0.0533 | Val Loss: 0.0536 | Time: 22.67s
Epoch 2/200 | Train Loss: 0.0471 | Val Loss: 0.0350 | Time: 22.81s
Epoch 3/200 | Train Loss: 0.0244 | Val Loss: 0.0189 | Time: 22.30s
Epoch 4/200 | Train Loss: 0.0195 | Val Loss: 0.0192 | Time: 22.41s
Epoch 5/200 | Train Loss: 0.0178 | Val Loss: 0.0162 | Time: 22.42s
Epoch 6/200 | Train Loss: 0.0165 | Val Loss: 0.0163 | Time: 22.58s
Epoch 7/200 | Train Loss: 0.0156 | Val Loss: 0.0146 | Time: 22.26s
Epoch 8/200 | Train Loss: 0.0148 | Val Loss: 0.0138 | Time: 22.07s
Epoch 9/200 | Train Loss: 0.0140 | Val Loss: 0.0136 | Time: 22.18s
Epoch 10/200 | Train Loss: 0.0135 | Val Loss: 0.0128 | Time: 22.32s
Epoch 11/200 | Train Loss: 0.0127 | Val Loss: 0.0126 | Time: 22.33s
Epoch 12/200 | Train Loss: 0.0123 | Val Loss: 0.0130 | Time: 22.71s
Epoch 13/200 | Train Loss: 0.0117 | Val Loss: 0.0125 | Time: 23.17s
Epoch 14/200 | Train Loss: 0.0115 |

### Calculate the performance metrics


In [18]:

import pandas as pd
from sklearn.metrics import r2_score


YEARS = [2021, 2022, 2023, 2024]
results = []

for k, test_year in enumerate(YEARS):

    target_file = np.load(f"/lustre/gpu-lustre/code/NeurOCAST_BiasCorrection_dev/NeurOCAST_BiasCorrection/example/Model_training/Test_target_{test_year}.npy")
    raw_file = np.load(f"/lustre/gpu-lustre/code/NeurOCAST_BiasCorrection_dev/NeurOCAST_BiasCorrection/example/Model_training/Test_raw_{test_year}.npy")
    prediction_file = np.load(f"/lustre/gpu-lustre/code/NeurOCAST_BiasCorrection_dev/NeurOCAST_BiasCorrection/example/Model_training/Test_prediction_{test_year}.npy")

    target = target_file.reshape(-1)
    raw = raw_file.reshape(-1)
    prediction = prediction_file.reshape(-1)

    # --- Calculate Raw Metrics ---
    rmse_raw = np.sqrt(np.mean((raw-target)**2))
    bias_raw = np.mean(raw-target)
    corr_raw = r2_score(target, raw) 

    # --- Calculate Prediction Metrics ---
    rmse_prediction = np.sqrt(np.mean((prediction-target)**2))
    bias_prediction = np.mean(prediction-target)
    corr_prediction = r2_score(target, prediction) 

    # --- Store Results for the Current Year ---
    results.append({
        'Test Year': test_year,
        'Metric': 'RMSE',
        'Raw': rmse_raw,
        'Bias Corrected': rmse_prediction
    })
    results.append({
        'Test Year': test_year,
        'Metric': 'Bias',
        'Raw': bias_raw,
        'Bias Corrected': bias_prediction
    })
    results.append({
        'Test Year': test_year,
        'Metric': 'Correlation',
        'Raw': corr_raw,
        'Bias Corrected': corr_prediction
    })


# --- Create and Print the Final Table ---
df = pd.DataFrame(results)

df_table = df.pivot_table(
    index=['Test Year', 'Metric'], 
    values=['Raw', 'Bias Corrected']
)

# Optional: Format the numbers to three decimal places
df_formatted = df_table.applymap(lambda x: f'{x:.3f}' if isinstance(x, (float, np.floating)) else x)

print(df_formatted)
    

                      Bias Corrected    Raw
Test Year Metric                           
2021      Bias                -0.016  0.032
          Correlation          0.931  0.833
          RMSE                 0.180  0.280
2022      Bias                 0.005  0.074
          Correlation          0.957  0.872
          RMSE                 0.126  0.217
2023      Bias                 0.030  0.197
          Correlation          0.951  0.834
          RMSE                 0.141  0.259
2024      Bias                 0.058  0.213
          Correlation          0.952  0.824
          RMSE                 0.137  0.261


/tmp/ipykernel_5826/638801577.py:58: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_formatted = df_table.applymap(lambda x: f'{x:.3f}' if isinstance(x, (float, np.floating)) else x)
